#### Import Modules

In [1]:
import os
import time
from pynput.keyboard import Key, Controller
import threading
import platform
import importlib

#### to_funct()
to_funct() - Converts students program to a callable function. 
Adds a function name - for calling, and return objects for validation of expected results
```
def <function_name>
    ~~~~~~~~
    student code here
    ~~~~~~~~
return <object(s)>
```
This modified program is written as a new temporary program for execution. 

In [2]:
def to_funct(fpath, file_name, return_results, cr_func_w_main=False):        
    '''
    function - reads into memory, the program undertests
        modifies the program by adding function definition and return statements, and writes
        to disk a new program (named _prg_undr_test.py) that can be imported
        and called by _prog_driver.py
        
        There are several different programs that this function parses: 
        | No Main Function & No Return Statement | 
        | Main Function    & No Return Statement | 
        | Main Function    & Return Statement | 
        
        In any case - to_funct() - needs to produce a new program with both a Main function and Return statement
            in order to programmatically execute the program. 
            
        Future consideration: 
            
            [code is too complex; should introduce regex to parsing logic, and reduce complexity, 
            and a number of unit tests]
            
            [ feb 2020 - added a unit test cell block, unit test folder, and beginning to
                add different programs that will exercise different paths of this function.]
        
        Maintained by kalen.howell@gmail.com 
    '''
    
    #print("working with file: " + file_name)
    
    file_name=os.path.join(fpath, file_name)
    
    if return_results != '':
        cr_return=True
    else:
        cr_return=False
    with open(file_name, "r") as in_file:
        buf = in_file.readlines()
    cnt = 0
    if cr_func_w_main == False and cr_return==True:
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                if cnt == 0:
                    line = "def do_work():\n\t" + line
                else:
                    line = "\t" + line
                cnt =+1
                out_file.write(line)
            out_file.write("\n\treturn " + return_results)
    elif cr_func_w_main == True:
        in_funct = False
        ret_writen = False
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                if cr_return ==True and "def " in line and in_funct == True:
                    line = "    return " + return_results + "\n" + line
                    in_funct = False
                    ret_writen = True
                if "def main():" in line:
                    line = "def do_work():\n"
                    in_funct = True
                if "main()" in line:
                    line = ''
                out_file.write(line)
            if ret_writen == False:
                out_file.write("\n    return " + return_results)
    else:
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                out_file.write(line)


#### Program Driver Cell Block
This code block - programmatically launches the temporary program (writen from above code), and conducts data-entry using provided input data. The student code processes that data according to the assignment requirements, and then returns the **actual results** to the calling function for verification (against **expected results**). 

In [3]:
# This Cell block represents: - a multithreaded application used to: 
# 1. launch the program under tests
# 2. drive data entry of the program under tests
# 3. compares expted results against actual results of the program under tests
# maintained by kalen.howell@gmail.com

def enter(data):
    '''
    function does data entry into program under tests.
    '''
    keyboard = Controller()
    
    for d in data:
        keyboard.press(d)
        keyboard.release(d)
    keyboard.press(Key.enter)
    keyboard.release(Key.enter)
    time.sleep(1)

def launch_typer(input_data):
    '''
    function does data entry into program under tests.
    '''
    time.sleep(1)
    for d in input_data:
        enter(d)

def launch_prg(prog, expected_results):
    '''
    launches program under tester, receives return values, and validates against expected results.
    '''
    results = prog.do_work()
    try:
        assert results == expected_results, "\n **FAILURE** \texpecting: " + str(expected_results) + "\n\trecieved: " + str(results) +"\n"
    except Exception as e: print(e)
    
def test(prog, input_data, expected_results):
    '''
    multithreaded function - 
        thread 1 - launches program under test, the other thread
        thread 2 - launches a typer programmer, which does data entry into the program under test
    '''
    prog_ut = threading.Thread(target=launch_prg, args=(prog, expected_results,),daemon=True)
    typer = threading.Thread(target=launch_typer, args=(input_data,), daemon=True)
    print('\n')
    
    prog_ut.start()
    typer.start()
    prog_ut.join()

def get_test_path(dir='test_dir'):
    '''
    Checks the host operating systems, and sets the working test path accordingly. 
    Default path - is working "test_dir" - the place to drop student programs to be tested. 
    ''' 
       
    win_fpath = 'C:\\Users\\KXH\\project_work\\python_work\\pygrader\\pygrader\\' + dir
    linux_fpath = '/Users/kalenhowellsr/Projects/code/pygrader/' + dir + '/'
    
    if platform.system() == "Windows":
        test_path = win_fpath
    else:
        test_path = linux_fpath  
    return test_path
        

#### Program Test Code
Serves a type of unit tests - to ensure any code changes made does not break pygrader

In [4]:
# ADDING DIFFERENT PROGRAMS AND ASSOCIATED TESTS - IN CASE UPDATES ARE NEEDED
#   TO PYGRADER CODE. ENSURE I DON'T BREAK ANYTHING WHEN MAKING CHANGES

# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

to_funct(fpath=get_test_path(dir="prog_test_dir"),
            file_name='Cookies.py',
            return_results='sugarNeeded, butterNeeded, flourNeeded',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver

import _prg_undr_test as prog
importlib.reload(prog) # flushes cache of previous test run's intermediate results

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['24'] , expected_results= (0.75, 0.5, 1.375))
test(prog, input_data=['65'] , expected_results= (2.03125, 1.3541666666666665, 3.723958333333333))
# the following produces an error for failure validation
test(prog, input_data=['65'] , expected_results= (2.03125, 1.3542666666666665, 3.723958333333333))

to_funct(fpath=get_test_path(dir="prog_test_dir"),
            file_name='Weight.py',
            return_results='weight',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)  # flushes cache of previous test run's intermediate results
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['6'] , expected_results= (58.800000000000004))
test(prog, input_data=['49'] , expected_results= (480.20000000000005))
test(prog, input_data=['50'] , expected_results= (490.00000000000006))
test(prog, input_data=['102'] , expected_results= (999.6))

to_funct(fpath=get_test_path(dir="prog_test_dir"),
            file_name='Ch3Exercise.py',
            return_results='None',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['90'] , expected_results= None)
test(prog, input_data=['81.6'] , expected_results= None)
test(prog, input_data=['70'] , expected_results= None)
test(prog, input_data=['62.8'] , expected_results= None)
test(prog, input_data=['55.5'] , expected_results= None)
test(prog, input_data=['102'] , expected_results= None)
test(prog, input_data=['-1'] , expected_results= None)

# Summation Test
to_funct(fpath=get_test_path(dir="prog_test_dir"),
            file_name='sum.py',
            return_results='sum',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['15', '45.29876','20000000','-3'] , expected_results= 20000060.29876)



Enter the number of cookies: 24
The amount of sugar needed is: 0.750 cups
The amount of butter needed is: 0.500 cups
The amount of flour needed is: 1.375 cups


Enter the number of cookies: 65
The amount of sugar needed is: 2.031 cups
The amount of butter needed is: 1.354 cups
The amount of flour needed is: 3.724 cups


Enter the number of cookies: 65
The amount of sugar needed is: 2.031 cups
The amount of butter needed is: 1.354 cups
The amount of flour needed is: 3.724 cups

 **FAILURE** 	expecting: (2.03125, 1.3542666666666665, 3.723958333333333)
	recieved: (2.03125, 1.3541666666666665, 3.723958333333333)



Enter the mass of an object: 6
The object is too light. The weight is 58.8 newtons


Enter the mass of an object: 49
The weight is 480.2 newtons


Enter the mass of an object: 50
The weight is 490.0 newtons


Enter the mass of an object: 102
The object is too heavy. The weight is 999.6 newtons


Enter numerical score: 90
A


Enter numerical score: 81.6
B


Enter numerical scor

#### Test: Cookies Assignment

In [ ]:
# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

to_funct(fpath=get_test_path(dir="test_dir"),
            file_name='Cookies.py',
            return_results='sugarNeeded, butterNeeded, flourNeeded',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)  # flushes cache of previous test run's intermediate results
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['24'] , expected_results= (0.75, 0.5, 1.375))
test(prog, input_data=['65'] , expected_results= (2.03125, 1.3541666666666665, 3.723958333333333))




#### Test:: Mass / Weight Tests Assignment

In [ ]:
#### pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

to_funct(fpath=get_test_path(dir="test_dir"),
            file_name='weight.py',
            return_results='weight',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['6'] , expected_results= (58.800000000000004))
test(prog, input_data=['49'] , expected_results= (480.20000000000005))
test(prog, input_data=['50'] , expected_results= (490.00000000000006))
test(prog, input_data=['102'] , expected_results= (999.6))


#### Test: Convert Score to Grades - (Ch3 Text Exercise)
Convert number grade to letter grade. 
This program require manual verifying of proper output. 

In [ ]:
# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

to_funct(fpath=get_test_path(dir="test_dir"),
            file_name='ch3.py',
            return_results='None',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['90'] , expected_results= None)
test(prog, input_data=['81.6'] , expected_results= None)
test(prog, input_data=['70'] , expected_results= None)
test(prog, input_data=['62.8'] , expected_results= None)
test(prog, input_data=['55.5'] , expected_results= None)
test(prog, input_data=['102'] , expected_results= None)
test(prog, input_data=['-1'] , expected_results= None)


###### Test: Sum user entered numbers - (Ch4 Ex. 8)
Promter user to enter a set of numbers, add, and output summation. 

Loop until the user enters a negative number, which stops the loop.

In [ ]:
# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

# Summation Test
to_funct(fpath=get_test_path(dir="test_dir"),
            file_name='sum.py',
            return_results='sum',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['15', '45.29876','20000000','-3'] , expected_results= 20000060.29876)


#### Compound Interest
Manually Visualize code

In [6]:
# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one

to_funct(fpath=get_test_path(dir="test_dir"),
            file_name='interest.py',
            return_results='None',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog
importlib.reload(prog)
os.remove('_prg_undr_test.py') # removing program after loading into memory

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['1500'] , expected_results= None)





Enter your deposit amount: 1500

Year	   Amount
----       ------
2018	  1,500.00
2019	  1,560.00
2020	  1,622.40
2021	  1,687.30
2022	  1,754.79
2023	  1,824.98
2024	  1,897.98
2025	  1,973.90
2026	  2,052.85
2027	  2,134.97
2028	  2,220.37
2029	  2,309.18
2030	  2,401.55
2031	  2,497.61
2032	  2,597.51
2033	  2,701.42
2034	  2,809.47
2035	  2,921.85
2036	  3,038.72
2037	  3,160.27
2038	  3,286.68

Total interest earned: 1,786.68


###### NEXT TEST


# 